In [149]:
!pip install python-Levenshtein
!pip install jaro-winkler
!pip install strsim


[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 42.4/42.4 kB 411.8 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [199]:
import pandas as pd 
import pickle
import geopandas as gpd
from Levenshtein import distance as lev
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
from similarity.normalized_levenshtein import NormalizedLevenshtein
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")
import jaro
normalized_levenshtein = NormalizedLevenshtein()

In [200]:
gsa=pd.read_csv(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - Reseaux de distibution\GSA_GSS 2020"+r'\GSA_2022_clean.csv')
gsa = gpd.GeoDataFrame(
    gsa, geometry=gpd.points_from_xy(gsa.X__WGS84_, gsa.Y__WGS84_)).set_crs("EPSG:4326").to_crs("2154")
data=pd.read_excel(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - IRIS\Geodata_IRIS_essentiels.xlsx")
list_departement=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Liste-Excel-des-departements-francais.xlsx")
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp').set_crs("EPSG:2154")
gss=pd.read_pickle(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - Reseaux de distibution\GSA_GSS 2020"+r'\GSA_GSS_actif_iris_2020_geo_gss.pkl')

In [201]:
# replace nan values of coordinates
nan_values_coordinates=gss[gss[['X__WGS84_','Y__WGS84_']].isna().any(axis=1)]
gss.loc[gss[['X__WGS84_','Y__WGS84_']].isna().any(axis=1),['X__WGS84_','Y__WGS84_']]=nan_values_coordinates.apply(lambda x :get_coordinates(x),axis=1).to_list()

In [202]:
gss=gss[~gss[['X__WGS84_','Y__WGS84_']].isna().any(axis=1)]
gss = gpd.GeoDataFrame(
    gss, geometry=gpd.points_from_xy(gss.X__WGS84_, gss.Y__WGS84_)).set_crs("EPSG:4326").to_crs("EPSG:2154")

In [203]:
logs=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\\logs.pkl","rb"))

In [204]:
def distance(x,gsa_name):
    normalized_levenshtein = NormalizedLevenshtein()
    min_distance=1
    for el in x.split(' '):
        for el_ in gsa_name.split(" "):
        
             if min_distance >normalized_levenshtein.distance(el.lower(),el_.lower()):
                min_distance=normalized_levenshtein.distance(el.lower(),el_.lower())
    return min_distance

In [205]:
def get_coordinates(x):
    try:
        return ox.geocode(str(x["N__Nom_de_voie"])+" "+str(x['CP']))
    except:
        return (None,None)

# Deduplication et correspondance GSA

In [206]:

def dedup_gsa(gsa_dep,path_to_save,shops):
    correspondance=pd.DataFrame(columns =["shop","gsa","distance","name_shop","name_gsa"])
    for i in range(len(gsa_dep)):
        #take all  the shops in circle of 100m  diamater
        #check the name of GSA and all other shops in this cirlce
        d=shops.distance(gsa_dep['geometry'].iloc[i]).sort_values()

        d=d[d<=100]



        #if there is some shops  nearby
        if len(d)>0:

            shops_nearby=shops.loc[d.index,:]

            shops_nearby_names=shops_nearby['name']
            gsa_name=gsa_dep.iloc[i]['Nom_Enseigne']


            #fill null values
            shops_nearby_names=shops_nearby_names.fillna("")

            #if one of the nearby shops has the same name as GSA/GSS then we associate them to each other
            a=np.array([i.strip().lower() for i in shops_nearby_names])
            a=np.array([i.replace('city',"").replace('market',"").replace('g20',"g 20") for  i in a])
            if gsa_name.strip().lower() in a:
                shop_id=shops_nearby_names[a==gsa_name.strip().lower()].index[0]
                shop_name=shops_nearby.loc[shop_id]['name']
                #name_dhopshops_nearby_names[a==(gsa_name.strip().lower())].to_numpy()[0])
                correspondance.loc[len(correspondance.index)]=[shop_id,i,0,shop_name,gsa_name]
                continue

            lev_distances=shops_nearby_names.apply(lambda x :distance(x,gsa_name))
            lev_distances=lev_distances[lev_distances<=0.25]

            #if there is no shop with the same name as GSA , we associate the GSA to the colsest shop 
            if len(lev_distances)>0:

                index_sim=lev_distances.index[lev_distances.argmin()]
                correspondance.loc[len(correspondance.index)]=[index_sim,i,d[index_sim],shops.loc[index_sim,"name"],gsa_name]
            elif str(shops.loc[d.index[d.argmin()],"name"]).strip().lower()=="NaN":
                closest_index=d.index[d.argmin()]
                correspondance.loc[len(correspondance.index)]=[closest_index,i,d[closest_index],shops.loc[closest_index,"name"],gsa_name]
    correspondance['name_shop']=correspondance['name_shop'].fillna("")
    
    #Remove duplicates
    #This happens when we associate the same shop to two or more different GSA

    counts=correspondance['shop'].value_counts()
    duplicated=[i   for i,x in correspondance.iterrows()  if x["shop"] in counts.index[counts>1]]
    duplicated_shops=correspondance[correspondance['shop'].isin(counts.index[counts>1])]
    all_index=[i for i in range(len(correspondance))]
    not_duplicated_index =[i for i in all_index if i not in duplicated]
    for index in list(set(duplicated_shops['shop'])):

        a=duplicated_shops[duplicated_shops["shop"]==index]

        a["edit_distance"]=a.apply(lambda x :normalized_levenshtein.distance(x['name_shop'].lower(),x['name_gsa'].lower()),axis = 1)
        a=a.sort_values("edit_distance")

        #the name of GSA and the shop is so close 
        #example  name_shop ==Bio C' Bon et name_gsa=INTERMARCHE EXPRESS or  BIO C BON  
        #we consider BIO C BON

        if len(a[a["edit_distance"]<=0.2])>0:

            not_duplicated_index.append(a[a["edit_distance"]<=0.2].index[0])
        
        else:
            #in this case, we consider the min distance
      
            min_index=a.sort_values("distance").index[0]

            not_duplicated_index.append(min_index)
    correspondance=correspondance.iloc[not_duplicated_index]
    
    pickle.dump(correspondance,open(path_to_save+r"\\association\\GSA_association.pkl","wb"))
    return correspondance

# Deduplication et correspondance GSS

In [207]:

def dedup_gss(gss_dep,path_to_save,shops):

    correspondance_gss=pd.DataFrame(columns =["shop","gss","distance","name_shop","name_gss"])
    for i in range(len(gss_dep)):
        #take all  the shops in circle of 100m  diamater
        #check the name of GSA and all other shops in this cirlce
        d=shops.distance(gss_dep['geometry'].iloc[i]).sort_values()
        
        d=d[d<=100]

        #if there is some shops in the proximity
        if len(d)>0:
            shops_nearby=shops.loc[d.index,:]

            shops_nearby_names=shops_nearby['name']
            gss_name=str(gss_dep.iloc[i]['Nom_Enseigne'])

            #fill null values
            shops_nearby_names=shops_nearby_names.fillna("")

            a=np.array([i.strip().lower() for i in shops_nearby_names])
            a=np.array([i.replace('city',"").replace('market',"").replace('g 20',"g20") for  i in a])
            if gss_name.strip().lower() in a:
                shop_id=shops_nearby_names[a==gss_name.strip().lower()].index[0]
                shop_name=shops_nearby.loc[shop_id]['name']
                #name_dhopshops_nearby_names[a==(gsa_name.strip().lower())].to_numpy()[0])
                correspondance_gss.loc[len(correspondance_gss.index)]=[shop_id,i,0,shop_name,gss_name]
                continue

            lev_distances=shops_nearby_names.apply(lambda x :distance(x,gss_name))
            lev_distances=lev_distances[lev_distances<=0.1]


            #if there is no shop with the same name as GSA , we associate the GSA to the colsest shop 
            if len(lev_distances)>0:

                index_sim=lev_distances.index[lev_distances.argmin()]
                correspondance_gss.loc[len(correspondance_gss.index)]=[index_sim,i,d[index_sim],shops.loc[index_sim,"name"],gss_name]
            elif str(shops.loc[d.index[d.argmin()],"name"]).strip().lower()=="nan":
                closest_index=d.index[d.argmin()]
                correspondance_gss.loc[len(correspondance_gss.index)]=[closest_index,i,d[closest_index],shops.loc[closest_index,"name"],gss_name]
    correspondance_gss['name_shop']=correspondance_gss['name_shop'].fillna("")
    counts=correspondance_gss['shop'].value_counts()
    duplicated=[i   for i,x in correspondance_gss.iterrows()  if x["shop"] in counts.index[counts>1]]
    duplicated_shops=correspondance_gss[correspondance_gss['shop'].isin(counts.index[counts>1])]
    all_index=[i for i in range(len(correspondance_gss))]
    not_duplicated_index =[i for i in all_index if i not in duplicated]
    for index in list(set(duplicated_shops['shop'])):
    
        a=duplicated_shops[duplicated_shops["shop"]==index]

        a["edit_distance"]=a.apply(lambda x :normalized_levenshtein.distance(x['name_shop'].lower(),x['name_gss'].lower()),axis = 1)
        a=a.sort_values("edit_distance")
    
        #the name of GSA and the shop is so close 
        #example  name_shop ==Bio C' Bon et name_gsa=INTERMARCHE EXPRESS or  BIO C BON  
        #we consider BIO C BON

        if len(a[a["edit_distance"]<=0.2])>0:

            not_duplicated_index.append(a[a["edit_distance"]<=0.2].index[0])
        
        else:
            #in this case, we consider the min distance
            min_index=a.sort_values("distance").index[0]

            not_duplicated_index.append(min_index)
    correspondance_gss=correspondance_gss.iloc[not_duplicated_index]
    pickle.dump(correspondance_gss,open(path_to_save+r"\\association\\GSS_association.pkl","wb"))
    return correspondance_gss

In [209]:
for i ,row in list_departement.iterrows():
    try:

        place_name=row["Département"].strip().lower()
        path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+r"\\"+row["Département"].lower()
        if place_name=="rhône":
            try:
                dep=int(row['N°'])
                gsa_dep=gsa[gsa['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
                gss_dep=gss[gss['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
            except:
                gsa_dep=gsa.copy()
                gss_dep=gss.copy()

            if len(gsa_dep)>0:
                shops=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+r"\\"+row["Département"].lower()+"\\raw data\\shops.pkl",'rb')).to_crs('EPSG:2154')
                correspondance=dedup_gsa(gsa_dep,path,shops)

                if place_name=='paris' or place_name=='haute-corse':
                    print(place_name)
                    print(len(correspondance))
                logs[place_name].update({"correspondance_gsa":len(correspondance)})
            if  len(gsa_dep)>0:
                shops=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+r"\\"+row["Département"].lower()+"\\raw data\\shops.pkl",'rb')).to_crs('EPSG:2154')
                correspondance_gss=dedup_gss(gss_dep,path,shops)
                logs[place_name].update({"correspondance_gss":len(correspondance_gss)})

    except:
        print(row["Département"])
        print("error")


        
    
pickle.dump(logs,open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\\logs.pkl","wb"))
